In [14]:
from dotenv import load_dotenv

load_dotenv()  # .env 파일을 환경 변수로 로드

True

In [41]:
"""
#5.7 LCEL Based Memory (07:38)


"""
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate,MessagesPlaceholder

In [47]:
llm = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True, # 메세지 형태로 출력
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

In [48]:
chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

In [49]:
def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [50]:
invoke_chain("My name is dooseop")

content='Hello Dooseop! How can I assist you today?'


In [51]:
invoke_chain("I live in Seoul")

content='Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or discuss about Seoul?'


In [53]:
invoke_chain("What is my name?")

content='Your name is Dooseop. How can I assist you further, Dooseop?'
